<a href="https://colab.research.google.com/github/Maxoo005/ml-wakacyjne-wyzwanie-2025/blob/main/4_Maksymilian_Kula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
!pip install langchain sentence-transformers faiss-cpu pypdf transformers torch langchain-community

IMPORTY

In [106]:
from pathlib import Path
from typing import List, Tuple, Optional
import json, math

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

Ładowanie pdf / splitter

In [107]:
pdf_files = [
    '/content/drive/MyDrive/pfkidoml/canabis wpływ na sen.pdf',
    '/content/drive/MyDrive/pfkidoml/czynniki wpływające na sen.pdf',
    '/content/drive/MyDrive/pfkidoml/Ashwagandha wpływa na sen.pdf'
]

CHUNK_SIZE = 1500
CHUNK_OVERLAP = 200
docs_raw = []
missing = []
for p in pdf_files:
    if Path(p).exists():
        docs_raw.extend(PyPDFLoader(p).load())
    else:
        missing.append(p)

if missing:
    print("Nie znaleziono plików (sprawdź dokładnie nazwy/ścieżki):")
    for m in missing: print(" -", m)

if not docs_raw:
    raise RuntimeError("Brak załadowanych PDF-ów. Popraw ścieżki w pdf_files.")

splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
documents = splitter.split_documents(docs_raw)
print(f"Załadowano {len(docs_raw)} rekordów, po podziale: {len(documents)} fragmentów.")

Załadowano 20 rekordów, po podziale: 66 fragmentów.


In [108]:
INDEX_DIR = "faiss_index_pl"
EMB_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
META_PATH = Path(INDEX_DIR) / "index_meta.json"

def get_embeddings():
    return HuggingFaceEmbeddings(
        model_name=EMB_MODEL,
        encode_kwargs={"normalize_embeddings": True}
    )

In [109]:
def build_faiss(docs):
    embeddings = get_embeddings()
    db = FAISS.from_documents(docs, embeddings)
    Path(INDEX_DIR).mkdir(parents=True, exist_ok=True)
    db.save_local(INDEX_DIR)
    with open(META_PATH, "w", encoding="utf-8") as f:
        json.dump({"emb_model": EMB_MODEL}, f, ensure_ascii=False, indent=2)
    print(" Zbudowano i zapisano nowy indeks FAISS.")
    return db

In [110]:
def load_or_build_faiss(docs):
    embeddings = get_embeddings()
    if Path(INDEX_DIR).exists() and META_PATH.exists():
        try:
            meta = json.loads(Path(META_PATH).read_text(encoding="utf-8"))
        except Exception:
            meta = {}
        if meta.get("emb_model") == EMB_MODEL:
            db = FAISS.load_local(INDEX_DIR, embeddings, allow_dangerous_deserialization=True)
            print("Załadowano istniejący indeks FAISS.")
            return db
        else:
            print("Wykryto inny model embedów w indeksie ->(przebudowa)")
            return build_faiss(docs)
    else:
        return build_faiss(docs)

db = load_or_build_faiss(documents)

Załadowano istniejący indeks FAISS.


model QA

In [111]:
QA_MODEL = "deepset/xlm-roberta-base-squad2"
qa_pipe = pipeline("question-answering", model=QA_MODEL, tokenizer=QA_MODEL)
print("Model QA gotowy.")

Device set to use cpu


Model QA gotowy.


Prosty system odpowiadania na pytania na podstawie dokumentów zapisanych w bazie wektorowej. Najpierw wyszukuje fragmenty najbardziej podobne do pytania, następnie łączy je z historią rozmowy i przekazuje do modelu QA, który generuje odpowiedź. Na końcu zwraca treść odpowiedzi wraz z oceną pewności i źródłem, z którego pochodzi.

In [112]:
TOP_K_DOCS = 4
MIN_COS_SIM = 0.20
MIN_QA_SCORE = 0.05
CONTEXT_LIMIT = 8000

def cosine_from_dist(dist: float) -> float:
    return 1.0 - (dist*dist)/2.0

def retrieve_with_scores(query: str, k: int = TOP_K_DOCS):
    return db.similarity_search_with_score(query, k=k)

def concat_context(chunks) -> str:
    parts, used = [], 0
    for d,_ in chunks:
        t = d.page_content.strip()
        if used + len(t) > CONTEXT_LIMIT:
            break
        parts.append(t); used += len(t)
    return "\n\n".join(parts)

def extract_source(best_doc) -> str:
    src = best_doc.metadata.get("source", "nieznane_źródło")
    page = best_doc.metadata.get("page")
    name = Path(src).name if src else "nieznane_źródło"
    return f"{name}, strona {int(page)+1}" if page is not None else name

def answer_question(query: str, chat_memory: List[Tuple[str, str]]):
    retrieved = retrieve_with_scores(query, k=TOP_K_DOCS)
    if not retrieved:
        return {"answer":"Brak dokumentów w bazie.",
                "score":0.0,"source":None,"guard":"no_docs","cos":0.0}

    best_doc, best_dist = min(retrieved, key=lambda x: x[1])
    best_cos = cosine_from_dist(best_dist)
    low_sim = best_cos < MIN_COS_SIM

    history_txt = "\n".join([f"Użytkownik: {q}\nAsystent: {a}" for q,a in chat_memory[-5:]])
    context = ""
    if history_txt:
        context += f"[Pamięć rozmowy]\n{history_txt}\n\n"
    context += "[Fragmenty z dokumentów]\n" + concat_context(retrieved)

    out = qa_pipe({"question": query, "context": context})
    ans = (out.get("answer") or "").strip()
    score = float(out.get("score", 0.0))

    guard = None
    if (not ans) or score < MIN_QA_SCORE:
        guard = "low_confidence" if not low_sim else "low_similarity"
        ans = "Nie znajduję pewnej odpowiedzi na podstawie dokumentów."

    return {"answer": ans,
            "score": score,
            "source": extract_source(best_doc),
            "guard": guard,
            "cos": best_cos}


In [113]:
chat_memory: List[Tuple[str, str]] = []

def ask(q: str):
    res = answer_question(q, chat_memory)
    if res["guard"] == "no_docs":
        print("BRAK DOKUMENTÓW")
    elif res["guard"] == "low_similarity":
        print("NISKA ZGODNOSC WYSZUKANYCH FRAGMENTOW Z PYTANIEM (cos≈{:.3f}).".format(res["cos"]))
    elif res["guard"] == "low_confidence":
        print("NISKA PEWNOSC MODELU QA(score≈{:.3f}).".format(res["score"]))

    print(f"\n Odpowiedź: {res['answer']}")
    if res["source"]: print(f"📎 Źródło: {res['source']}")
    print(f" QA score: {res['score']:.3f} |  cos≈{res['cos']:.3f}")

    chat_memory.append((q, res["answer"]))
    return res

print("QA + FAISS gotowe. Użyj: ask('Twoje pytanie...')")

QA + FAISS gotowe. Użyj: ask('Twoje pytanie...')


In [114]:
def debug_retrieve(q: str, k: int = 3):
    hits = db.similarity_search_with_score(q, k=k)
    for i,(doc,dist) in enumerate(hits,1):
        cos = cosine_from_dist(dist)
        print(f"\n=== HIT {i} | cos≈{cos:.3f} | {doc.metadata.get('source')} p.{doc.metadata.get('page')}")
        print(doc.page_content[:600].replace("\n"," ") + " …")
    return hits

In [115]:
ask("Jaki związek chemiczny w ashwagandzie odpowiada za efekt nasenny")


 Odpowiedź: glikol trietylenowy.
📎 Źródło: Ashwagandha wpływa na sen.pdf, strona 2
 QA score: 1.880 |  cos≈0.765


{'answer': 'glikol trietylenowy.',
 'score': 1.8802032285893802,
 'source': 'Ashwagandha wpływa na sen.pdf, strona 2',
 'guard': None,
 'cos': np.float32(0.7646045)}

In [116]:
ask("Czy ashwagandha wpływa na sen")


 Odpowiedź: bezsenność;
📎 Źródło: Ashwagandha wpływa na sen.pdf, strona 2
 QA score: 0.130 |  cos≈0.947


{'answer': 'bezsenność;',
 'score': 0.1296029942750465,
 'source': 'Ashwagandha wpływa na sen.pdf, strona 2',
 'guard': None,
 'cos': np.float32(0.94667697)}

In [117]:
ask("Jak działa CBD w małych i dużych dawkach")

NISKA PEWNOSC MODELU QA(score≈0.010).

 Odpowiedź: Nie znajduję pewnej odpowiedzi na podstawie dokumentów.
📎 Źródło: canabis wpływ na sen.pdf, strona 5
 QA score: 0.010 |  cos≈0.701


{'answer': 'Nie znajduję pewnej odpowiedzi na podstawie dokumentów.',
 'score': 0.00997854769229889,
 'source': 'canabis wpływ na sen.pdf, strona 5',
 'guard': 'low_confidence',
 'cos': np.float32(0.70142066)}

In [118]:
ask("Ile osób wzięło udział w badaniu")


 Odpowiedź: 122 osoby
📎 Źródło: czynniki wpływające na sen.pdf, strona 2
 QA score: 0.598 |  cos≈0.795


{'answer': '122 osoby',
 'score': 0.5977151691913605,
 'source': 'czynniki wpływające na sen.pdf, strona 2',
 'guard': None,
 'cos': np.float32(0.7952308)}

Zbudowałem asystenta który posiada:
1. baze wektorową FAISS(z zapisem i odczytem)
2. embeddingi z modelu paraphrase-multilingual-MiniLM-L12-v2
3. QA z deepset/xlm-roberta-base-squad2
4. pamięć rozmowy
5. mechanizm anty-halucynacji
6. mechanizm który podaje dokladne źródło nazwa pliiku + strona
7. próbowalem pobawic sie z multispanem
8. QA zwraca sensowne odpowiedzi, informuje o prawdopodobienstwie i pewnosci, nie wymysla odpowiedzi


Wnioski z 4 pytań testowych
- odpowiedz na pierwsze pytanie zwróciała dokładną odpowiedź jak i wskazanie strony i pliku
- odpowiedz na 2 pytanie, bardzo dobre dopasowanie, odpowiedz trafna
- odpowiedz na 3 pytanie "BRAK" działa anty-haluna, niska pewnosc (score=0,02), sensowny retrieval(cos=0,7) sytem nie podaje odpowiedzi w prost, nie zmysla
- odpowiedz na 4 pytanie, poradzil sobie bez problemu i wyciągnął odpowiednie liczby

ostatnie podsumowanie
- Gdy cosinus podobieństwa jest wysoki (≥~0.75) i w tekście jest zwarte zdanie lub liczba, dostaję poprawną odpowiedź + cytowanie.

- Gdy cos jest OK, ale QA-score niski, wchodzi „bezpieczne nie wiem” — to jest mój bezpiecznik przed halucynacją.

- Pytania ekstrakcyjne (kto? co? ile? jaki związek?) działają najlepiej; pytania złożone/porównawcze („małe vs duże dawki CBD”) wymagają albo innego sformułowania, albo większego kontekstu.